In [2]:
from oligopoolio import * 

In [15]:
import pyswarms as ps
import numpy as np
import itertools
from primer3 import calc_hairpin, calc_homodimer
from functools import partial

def gc_content(seq):
    return (seq.count('G') + seq.count('C')) / len(seq)

def objective_function(x, seq):
    penalties = []
    for row in x:
        # The row contains the specified cut sites for the sequence 
        cut_sites = row
        # 
        results = check_secondary_structure(seq)
        homodimer_tm = results['homodimer']['homodimer_dg']
        hairpin_tm = results['hairpin']['hairpin_dg']
        
        # Penalize if Tm deviates from 60
        primer_tm = primer3.bindings.calcTm(seq)

        tm_penalty = np.abs(primer_tm - tm_optimal)
        
        # Penalize if GC content is far from 0.5
        gc_penalty = np.abs(gc - gc_optimal)*30
        # Example: simple penalty for homopolymer runs
        homodimer_tm = 0 if homodimer_tm > 0 else np.abs(homodimer_tm)
        penalties.append(tm_penalty + gc_penalty + homodimer_tm + clamp_penalty)
    return np.array(penalties)

    
def check_secondary_structure(sequence, temp=55):
    """
    Check secondary structures like hairpins and homodimers in a given primer sequence.
    
    Args:
        sequence (str): The DNA sequence of the primer to analyze.
        
    Returns:
        dict: Results for hairpin and homodimer properties.
    """
    try:
        # Check for hairpin structure
        hairpin_result = calc_hairpin(sequence, temp_c=temp)
        hairpin_info = {
            "hairpin_found": hairpin_result.structure_found,
            "hairpin_tm": hairpin_result.tm,
            "hairpin_dg": hairpin_result.dg/1000,
            "hairpin_dh": hairpin_result.dh/1000,
            "hairpin_ds": hairpin_result.ds,
        }

        # Check for homodimer structure
        homodimer_result = calc_homodimer(sequence, temp_c=temp)
        homodimer_info = {
            "homodimer_found": homodimer_result.structure_found,
            "homodimer_tm": homodimer_result.tm,
            "homodimer_dg": homodimer_result.dg/1000,
            "homodimer_dh": homodimer_result.dh/1000,
            "homodimer_ds": homodimer_result.ds,
        }
    except Exception as e:
        u.warn_p([f"Warning: issue with secondary structure check. ", sequence, e])
        hairpin_info = {"hairpin_found": False, "hairpin_tm": None, "hairpin_dg": None, "hairpin_dh": None, "hairpin_ds": None}
        homodimer_info = {"homodimer_found": False, "homodimer_tm": None, "homodimer_dg": None, "homodimer_dh": None, "homodimer_ds": None}
    # Combine results
    return {"hairpin": hairpin_info, "homodimer": homodimer_info}

In [ ]:
from sciutil import SciUtil
import random
import Levenshtein
from collections import defaultdict

u = SciUtil()

gc_optimal = 0.5
tm_optimal = 67
structure_dg = 0 # aim for greater than 0

# Probably best to use something like pyswarm 
def objective_function(x, seq):
    penalties = []
    for row in x:
        # The row contains the specified cut sites for the sequence 
        cut_sites = row
        # 
        results = check_secondary_structure(seq)
        homodimer_tm = results['homodimer']['homodimer_dg']
        hairpin_tm = results['hairpin']['hairpin_dg']
        
        # Penalize if Tm deviates from 60
        primer_tm = primer3.bindings.calcTm(seq)

        tm_penalty = np.abs(primer_tm - tm_optimal)
        
        # Penalize if GC content is far from 0.5
        gc_penalty = np.abs(gc - gc_optimal)*30
        # Example: simple penalty for homopolymer runs
        homodimer_tm = 0 if homodimer_tm > 0 else np.abs(homodimer_tm)
        penalties.append(tm_penalty + gc_penalty + homodimer_tm + clamp_penalty)
    return np.array(penalties)
    
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9} 
seq_ids = defaultdict(list)
all_sequences = defaultdict(list)

for seq in seqs:
    seq_len = len(seq)
    num_splits = int(math.ceil(seq_len / min_segment_length))
    # Make sure it is even
    if num_splits % 2 == 0:
        num_splits += 1
        
    bounds = []

    optimizer = ps.single.GlobalBestPSO(
        n_particles=50,
        dimensions=primer_len,
        options=options,
        bounds=bounds
    )
    
    best_cost, best_pos = optimizer.optimize(partial(objective_function, seq=sample), iters=200)
    
except:
    print(sample)

In [168]:
import numpy as np
import pyswarms as ps
from functools import partial



# Helper to compute fragment start-end pairs
def get_fragments(cuts, seq_len, num_fragments):
    cuts = np.sort(np.round(cuts).astype(int))
    starts = [0] + list(cuts)
    fragments = []

    for i in range(num_fragments):
        start = starts[i]
        # estimate end based on next start minus overlap
        if i < num_fragments - 1:
            next_start = starts[i + 1]
            end = next_start + max_overlap
        else:
            end = seq_len
        end = min(end, seq_len)
        fragments.append((start, end))
    return fragments

# Objective: minimize fragment length variance, and penalize invalid overlap
def objective_function(x, seq_len, num_fragments, seq, tm_optimal=62):
    penalties = []
    for particle in x:
        fragments = get_fragments(particle, seq_len, num_fragments)
        frag_lengths = [end - start for start, end in fragments]
        
        # Overlap check
        olap_penalty = 0
        # Also keep track of the overlaps so that we can select ones which aren't similar
        overlaps = [] 
        temp_penalty = 0
        seq_len_penalty = 0
        homodimer_penalty = 0
        repeat_penalty = 0
        for i in range(len(fragments) - 1):
            start1, end1 = fragments[i]
            start2, end2 = fragments[i + 1]
            overlap = end1 - start2
            # Check the temperature of the overlap and the dimerization 
            results = check_secondary_structure(seq[start2: end1])
            homodimer_tm = results['homodimer']['homodimer_dg']
            hairpin_tm = results['hairpin']['hairpin_dg']
            
            # Penalize if Tm deviates from 60
            primer_tm = primer3.bindings.calcTm(seq[start2: end1])
            
            tm_penalty = np.abs(primer_tm - tm_optimal)
            if overlap < min_overlap or overlap > max_overlap:
                olap_penalty += 10 * abs(overlap - (min_overlap + max_overlap)/2)

            # Have a tolerance for the TM (i.e. is 6C ok?)
            if tm_penalty > 5:
                temp_penalty += 2*tm_penalty
            else:
                temp_penalty += tm_penalty
            if end1 - start1 > max_seq_len:# or end1 - start1 < min_seq_len:
                seq_len_penalty += 10*abs(max_seq_len- (end1 - start1))
            elif end1 - start1 < min_seq_len: 
                seq_len_penalty += 10*abs(min_seq_len- (end1 - start1))
            if end2 - start2 > max_seq_len:# or end1 - start1 < min_seq_len:
                seq_len_penalty += 10*abs(max_seq_len- (end2 - start2))
            elif end2 - start2 < min_seq_len: 
                seq_len_penalty += 10*abs(min_seq_len- (end2 - start2)) 
            if homodimer_tm < -3:
                # Add an extra penalty if it's outside the range
                homodimer_penalty += 2*abs(homodimer_tm)
            else:
                homodimer_penalty += abs(homodimer_tm)
            overlap_seq = seq[start2: end1]
            if overlap_seq[:3] in ['AAA', 'TTT', 'CCC', 'GGG'] or overlap_seq[-3:] in ['AAA', 'TTT', 'CCC', 'GGG']:
                repeat_penalty += 20
            overlaps.append(seq[start2: end1])
        
        # Want to do levenstein distance and make sure there is at least an edit distance of 8 otherwise these are too similar.
        min_dist = 100000
        for i, x in enumerate(overlaps):
            for j, y in enumerate(overlaps):
                if i != j:
                    dist = Levenshtein.distance(x, y)
                    if dist < min_dist:
                        min_dist = dist
        dist_penalty = 27 - min_dist
        if min_dist < 8:
            dist_penalty += 27
        # Variance penalty
        mean_len = np.mean(frag_lengths)
        variance = np.sqrt(np.sum([(l - mean_len)**2 for l in frag_lengths]))
        penalties.append(variance + olap_penalty + homodimer_penalty + temp_penalty + seq_len_penalty + dist_penalty + repeat_penalty)
    return np.array(penalties)


In [169]:
import Levenshtein

# Parameters
seq = "gaaataattttgtttaactttaagaaggagatatacatATGCGCAAGCTTTCTCGTTGTGAGGACGGCTACGTCAAGATTCAGGGGATCTACATTTACTATAAAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCACGGTGGACCCGGCATGAGCCACGACTACTTGCTTTCCCTGACCGATTTAGCTGAGAAGGGGATTACGGTTTTATTCTATGACCAATTCGGCTGTGGACGTTCAGAAGAACCTGAGAAGGAGAAATTTACGATCGACTATGGAGTGGAAGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTATTCCTGATGGGATCGTCCTATGGTGGGGCTTTAGCTCTGGCCTACGCTGTTAAGTATCAAGCACATTTGAAGGGCCTGATCATCTCGGGCGGATTATCATCTGTGCCTCTGACAGTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACCGCAACGCGATCCGCAAGTATGGCGAAGTGGGGGATTATCAGAACCCTGAATACCAGGAGGCTGTGAACTATTTCTATCACCAGCATTTGTTACGCTCTGAAGACTGGCCCCCCGAGGTTTTGAAAAGCCTGGAGTACGCAGAAGAGCGCAATGTGTACCGCACAATGAACGGCCCCAATGAATTTACAATCACTGGAACTATCCGCGACTGGGATATCACCGATAAGATCGGGATTATCAGTGTTCCTACTCTTATTACGGTCGGTGAGTTTGACGAGGTAACGCAAAATGTCGCAGAAGTTATCCACTCCAAAATCGATAATTCCCAGTTAATTGTATTTAAAGCATGTAGCCACCTGACCATGTGGGAGGATCGTGATGAGTACAACCGCATTTTATTACAGTTCATCGAAAAGAACATTCTCGAGCACCACCACCACCACCACTGAgatccggc"
seq_len = len(seq)
min_overlap = 16
max_overlap = 27
max_seq_len = 130
min_seq_len = 80
optimal_seq_len = 100

num_fragments = int(math.ceil(seq_len / optimal_seq_len))
# Make sure it is even
if num_fragments % 2 == 1:
    num_fragments += 1
        
# Define bounds for cuts
num_cuts = num_fragments - 1
lower_bounds = np.arange(1, num_cuts + 1) * 2  # avoid first char
upper_bounds = np.full(num_cuts, seq_len - 2)

# Run PSO
bounds = (lower_bounds, upper_bounds)
optimizer = ps.single.GlobalBestPSO(
    n_particles=150,
    dimensions=num_cuts,
    options={'c1': 0.5, 'c2': 0.5, 'w': 0.7},
    bounds=bounds
)

best_cost, best_pos = optimizer.optimize(partial(objective_function, seq_len=seq_len, num_fragments=num_fragments, seq=seq), iters=100)

# Decode result
cuts = np.sort(np.round(best_pos).astype(int))
fragments = get_fragments(cuts, seq_len, num_fragments)
fragment_strings = [seq[start:end] for start, end in fragments]

print("Best cut sites:", fragments)
print("Fragments:", fragment_strings)

2025-06-24 11:32:36,157 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.5, 'w': 0.7}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|100/100, best_cost=82.4
2025-06-24 11:33:01,879 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 82.36328492184145, best pos: [204.04128056 102.4249867  644.49720023 380.0298606  740.12362654
 294.55862508 552.4081172  454.93127964 837.87533045]


Best cut sites: [(0, np.int64(129)), (np.int64(102), np.int64(231)), (np.int64(204), np.int64(322)), (np.int64(295), np.int64(407)), (np.int64(380), np.int64(482)), (np.int64(455), np.int64(579)), (np.int64(552), np.int64(671)), (np.int64(644), np.int64(767)), (np.int64(740), np.int64(865)), (np.int64(838), 958)]
Fragments: ['gaaataattttgtttaactttaagaaggagatatacatATGCGCAAGCTTTCTCGTTGTGAGGACGGCTACGTCAAGATTCAGGGGATCTACATTTACTATAAAGTGTGCAAAGCCGAGAATGAGAAAG', 'AAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCACGGTGGACCCGGCATGAGCCACGACTACTTGCTTTCCCTGACCGATTTAGCTGAGAAGGGGATTACGGTTTTATTCTATGACC', 'AGGGGATTACGGTTTTATTCTATGACCAATTCGGCTGTGGACGTTCAGAAGAACCTGAGAAGGAGAAATTTACGATCGACTATGGAGTGGAAGAAGCAGAAGCGGTGAAAAAAAACAT', 'AGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTATTCCTGATGGGATCGTCCTATGGTGGGGCTTTAGCTCTGGCCTACGCTGTTAAGTATCAAGCACAT', 'GCCTACGCTGTTAAGTATCAAGCACATTTGAAGGGCCTGATCATCTCGGGCGGATTATCATCTGTGCCTCTGACAGTAAAAGAAATGCAGCGCTTGATTGAC', 'GTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACCGCAACGCGATCCGCAA

In [170]:
seq = "ATGCGCAAGCTTTCTCGTTGTGAGGACGGCTACGTCAAGATTCAGGGGATCTACATTTACTATAAAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCACGGTGGACCCGGCATGAGCCACGACTACTTGCTTTCCCTGACCGATTTAGCTGAGAAGGGGATTACGGTTTTATTCTATGACCAATTCGGCTGTGGACGTTCAGAAGAACCTGAGAAGGAGAAATTTACGATCGACTATGGAGTGGAAGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTATTCCTGATGGGATCGTCCTATGGTGGGGCTTTAGCTCTGGCCTACGCTGTTAAGTATCAAGCACATTTGAAGGGCCTGATCATCTCGGGCGGATTATCATCTGTGCCTCTGACAGTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACCGCAACGCGATCCGCAAGTATGGCGAAGTGGGGGATTATCAGAACCCTGAATACCAGGAGGCTGTGAACTATTTCTATCACCAGCATTTGTTACGCTCTGAAGACTGGCCCCCCGAGGTTTTGAAAAGCCTGGAGTACGCAGAAGAGCGCAATGTGTACCGCACAATGAACGGCCCCAATGAATTTACAATCACTGGAACTATCCGCGACTGGGATATCACCGATAAGATCGGGATTATCAGTGTTCCTACTCTTATTACGGTCGGTGAGTTTGACGAGGTAACGCAAAATGTCGCAGAAGTTATCCACTCCAAAATCGATAATTCCCAGTTAATTGTATTTAAAGCATGTAGCCACCTGACCATGTGGGAGGATCGTGATGAGTACAACCGCATTTTATTACAGTTCATCGAAAAGAACATTCTCGAGCACCACCACCACCACCACTGA"

In [171]:
# Check the overlaps by making the plasmid map
from oligopoolio import * 
oligos = fragment_strings
output_file = 'test.gb'
prev_oligo = None
genbank_file = 'base-pet22b-base-anm.gb'
seq_id = 'test'
# Add in the optimzed sequence
translation_label = f"Insert_{seq_id}"
reverse = False  # Set to True for reverse feature, False for forward
insert_position = 5193
reverse = False
record = insert_sequence_with_translation(genbank_file, None, insert_position, seq, translation_label, reverse)
bsa = False
rows = []
# If a genbank file was provided also just add in the new sequnece
if len(oligos) > 0:
    for i, oligo in enumerate(oligos):
        seq = oligo
        # CHeck that there is an overlap with the previous sequence and that it is not too short
        # Also make sure we swap the directions of the oligos so they automatically anneal
        # Also assert that the start is a methionine (and if not warn it... )
        primer_overlap = None
        primer_tm = None
        primer_len = None
        homodimer_tm = None
        hairpin_tm = None
        if prev_oligo:  
            # Get the overlap with the previous sequence
            match = SequenceMatcher(None, prev_oligo, seq).find_longest_match()
            primer_overlap = prev_oligo[match.a:match.a + match.size]
            # Analyze the primer sequence
            results = check_secondary_structure(primer_overlap)
            homodimer_tm = results['homodimer']['homodimer_dg']
            hairpin_tm = results['hairpin']['hairpin_dg']
            primer_tm = primer3.bindings.calcTm(primer_overlap)
            primer_len = len(primer_overlap)

        prev_oligo = seq
        orig_seq = seq
        strand = 1
        # Changed this to be the opposite To
        if bsa == True:
            if i % 2 == 1 and i != 0:
                seq = str(Seq(seq).reverse_complement())
                strand = -1
        else:
            if i % 2 == 0:
                seq = str(Seq(seq).reverse_complement())
                strand = -1
        oligo_tm = primer3.bindings.calcTm(seq)
        #if genbank_file:
        insert_features_from_oligos(record, f"{seq_id}_oligo_{i}", orig_seq, strand, oligo_tm, None)
        rows.append([seq_id, oligo, seq, len(seq), oligo_tm, strand, primer_overlap, primer_tm, primer_len, homodimer_tm, hairpin_tm, oligo[2]])
    if genbank_file:
        output_file = genbank_file.replace('.', f'_{seq_id}.')
        if genbank_file:
            record.name = seq_id  # type: ignore 
            SeqIO.write(record, f'{output_file}', "genbank")

In [172]:
oligo_df = pd.DataFrame(rows, columns=["id", "oligo_id", "oligo_sequence", "oligo_length",  "oligo_tm", "strand","primer_overlap_with_previous", "overlap_tm_5prime", "overlap_length", 
                                            "overlap_homodimer_dg", "overlap_hairpin_dg", "original_sequence"])
oligo_df

,id,oligo_id,oligo_sequence,oligo_length,oligo_tm,strand,primer_overlap_with_previous,overlap_tm_5prime,overlap_length,overlap_homodimer_dg,overlap_hairpin_dg,original_sequence
0,test,gaaataattttgtttaactttaagaaggagatatacatATGCGCAA...,CTTTCTCATTCTCGGCTTTGCACACTTTATAGTAAATGTAGATCCC...,129,74.650485,-1,None,NaN,NaN,NaN,NaN,a
1,test,AAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCA...,AAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCA...,129,81.704538,1,AAGTGTGCAAAGCCGAGAATGAGAAAG,62.231401,27.0,-2.210118,0.000000,G
2,test,AGGGGATTACGGTTTTATTCTATGACCAATTCGGCTGTGGACGTTC...,ATGTTTTTTTTCACCGCTTCTGCTTCTTCCACTCCATAGTCGATCG...,118,78.597154,-1,AGGGGATTACGGTTTTATTCTATGACC,58.534689,27.0,-0.959718,0.000000,G
3,test,AGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTA...,AGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTA...,112,79.263673,1,AGAAGCAGAAGCGGTGAAAAAAAACAT,60.752486,27.0,-1.666098,0.000000,A
4,test,GCCTACGCTGTTAAGTATCAAGCACATTTGAAGGGCCTGATCATCT...,GTCAATCAAGCGCTGCATTTCTTTTACTGTCAGAGGCACAGATGAT...,102,79.737765,-1,GCCTACGCTGTTAAGTATCAAGCACAT,61.495744,27.0,-1.336169,0.000000,C
5,test,GTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACC...,GTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACC...,124,80.997246,1,GTAAAAGAAATGCAGCGCTTGATTGAC,60.276860,27.0,-4.656577,0.000000,A
6,test,ACCAGGAGGCTGTGAACTATTTCTATCACCAGCATTTGTTACGCTC...,GTTCATTGTGCGGTACACATTGCGCTCTTCTGCGTACTCCAGGCTT...,119,81.968434,-1,ACCAGGAGGCTGTGAACTATTTCTATC,60.117426,27.0,-2.838108,0.000000,C
7,test,GAGCGCAATGTGTACCGCACAATGAACGGCCCCAATGAATTTACAA...,GAGCGCAATGTGTACCGCACAATGAACGGCCCCAATGAATTTACAA...,123,80.781631,1,GAGCGCAATGTGTACCGCACAATGAAC,65.174207,27.0,-4.342888,-0.226057,G
8,test,AGTGTTCCTACTCTTATTACGGTCGGTGAGTTTGACGAGGTAACGC...,ATGGTCAGGTGGCTACATGCTTTAAATACAATTAACTGGGAATTAT...,125,78.231318,-1,AGTGTTCCTACTCTTATTACGGTCGGT,60.971734,27.0,0.392122,0.000000,T
9,test,ATTTAAAGCATGTAGCCACCTGACCATGTGGGAGGATCGTGATGAG...,ATTTAAAGCATGTAGCCACCTGACCATGTGGGAGGATCGTGATGAG...,120,78.734479,1,ATTTAAAGCATGTAGCCACCTGACCAT,60.985601,27.0,-1.096238,0.000000,T


In [153]:
overlaps = [x for x in oligo_df['primer_overlap_with_previous'].values if x != None]
overlaps

['AAGTGTGCAAAGCCGAGAATGAGAAAG',
 'AGGGGATTACGGTTTTATTCTATGACC',
 'AGAAGCAGAAGCGGTGAAAAAAAACAT',
 'TAGCTCTGGCCTACGCTGTTAAGTATC',
 'CGGATTATCATCTGTGCCTCTGACAGT',
 'TGGGGGATTATCAGAACCCTGAATACC',
 'CTGGAGTACGCAGAAGAGCGCAATGTG',
 'GATCGGGATTATCAGTGTTCCTACTCT',
 'GTTAATTGTATTTAAAGCATGTAGCCA']

In [154]:
# Want to do levenstein distance and make sure there is at least an edit distance of 8 otherwise these are too similar.
min_dist = 100000
for i, x in enumerate(overlaps):
    for j, y in enumerate(overlaps):
        if i != j:
            dist = Levenshtein.distance(x, y)
            if dist < min_dist:
                min_dist = dist
dist_penalty = 0
if dist < 8:
    dist_penalty = 100
    print(min_dist)

In [155]:
min_dist

12

In [ ]:
import dnaweaver as dw
import time
import dnachisel as dnachisel 
from sciutil import SciUtil
from Bio.Seq import Seq
from difflib import SequenceMatcher
import primer3
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import math
from dnachisel import *

u = SciUtil()

from primer3 import calc_hairpin, calc_homodimer


from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

def insert_sequence_with_translation(input_file, output_file, insert_position, new_sequence, translation_label, reverse=False):
    """
    Insert a new sequence at a specific position in a GenBank file, add a translation annotation,
    and adjust existing annotations to avoid overlap.

    Args:
        input_file (str): Path to the original GenBank file.
        output_file (str): Path to save the modified GenBank file.
        insert_position (int): Position to insert the new sequence (0-based).
        new_sequence (str): The DNA sequence to insert.
        translation_label (str): Label for the translation feature.
        reverse (bool): Whether the feature should be on the reverse strand.
    """
    # Read the original GenBank file
    record = SeqIO.read(input_file, "genbank")
    
    # Insert the new sequence at the specified position
    if reverse:
        new_sequence = str(Seq(new_sequence).reverse_complement())  # Reverse complement the sequence if needed
    record.seq = record.seq[:insert_position] + Seq(new_sequence) + record.seq[insert_position:]
    
    # Adjust existing annotations to avoid overlap
    inserted_length = len(new_sequence)
    for feature in record.features:
        if feature.location.start >= insert_position:
            feature.location = FeatureLocation(
                feature.location.start + inserted_length,
                feature.location.end + inserted_length,
                strand=feature.location.strand
            )
    
    # Create the feature label
    strand_label = " (reverse)" if reverse else " (forward)"
    full_label = translation_label + strand_label
    
    # Add a feature for the inserted sequence
    start = insert_position
    end = insert_position + len(new_sequence)
    feature = SeqFeature(
        location=FeatureLocation(start, end, strand=-1 if reverse else 1),
        type="CDS",  # CDS type for coding sequences
        qualifiers={
            "label": full_label,
            "translation": Seq(new_sequence).translate(table=11)  # Translation for the sequence
        }
    )
    record.features.append(feature)
    # Save the modified GenBank file
    if output_file:
        SeqIO.write(record, output_file, "genbank")
        print(f"Updated GenBank file saved as {output_file}")
    
    return record

def insert_features_from_oligos(record, seq_id, seq, strand, tm, output_file):
    """
    Insert features into a GenBank file based on oligo_df.

    Args:
        genbank_file (str): Path to the original GenBank file.
        output_file (str): Path to save the updated GenBank file.
        oligo_df (pd.DataFrame): DataFrame with oligo information. 
                                 Expected columns: ['id', 'oligo_id', 'oligo_sequence', 'oligo_length', ...].
    """
    # Iterate through the oligo DataFrame to add features
    start = record.seq.find(seq.upper())
    if start == -1:
        print(f"Warning: Oligo sequence {seq_id} not found in the GenBank sequence.")
    
    end = start + len(seq)
    feature = SeqFeature(
        location=FeatureLocation(start, end, strand=strand),
        type="misc_feature",
        qualifiers={
            "label": f"{seq_id} {'(reverse)' if strand == -1 else '(forward)'}",
            "note": f"Length: {len(seq)}, TM: {tm}"
        }
    )
    record.features.append(feature)
    
    # Save the updated GenBank file
    if output_file:
        SeqIO.write(record, output_file, "genbank")
        print(f"Updated GenBank file saved as {output_file}")
    return record
    
def check_secondary_structure(sequence, temp=55):
    """
    Check secondary structures like hairpins and homodimers in a given primer sequence.
    
    Args:
        sequence (str): The DNA sequence of the primer to analyze.
        
    Returns:
        dict: Results for hairpin and homodimer properties.
    """
    try:
        # Check for hairpin structure
        hairpin_result = calc_hairpin(sequence, temp_c=temp)
        hairpin_info = {
            "hairpin_found": hairpin_result.structure_found,
            "hairpin_tm": hairpin_result.tm,
            "hairpin_dg": hairpin_result.dg/1000,
            "hairpin_dh": hairpin_result.dh/1000,
            "hairpin_ds": hairpin_result.ds,
        }

        # Check for homodimer structure
        homodimer_result = calc_homodimer(sequence, temp_c=temp)
        homodimer_info = {
            "homodimer_found": homodimer_result.structure_found,
            "homodimer_tm": homodimer_result.tm,
            "homodimer_dg": homodimer_result.dg/1000,
            "homodimer_dh": homodimer_result.dh/1000,
            "homodimer_ds": homodimer_result.ds,
        }
    except Exception as e:
        u.warn_p([f"Warning: issue with secondary structure check. ", sequence, e])
        hairpin_info = {"hairpin_found": False, "hairpin_tm": None, "hairpin_dg": None, "hairpin_dh": None, "hairpin_ds": None}
        homodimer_info = {"homodimer_found": False, "homodimer_tm": None, "homodimer_dg": None, "homodimer_dh": None, "homodimer_ds": None}
    # Combine results
    return {"hairpin": hairpin_info, "homodimer": homodimer_info}

def build_oligos(seq_id: str, sequence: str, output_directory: str, min_gc=0.3, max_gc=0.7, min_tm=55, max_tm=70, min_segment_length=40, max_segment_length=100, max_length=1500):
    """ Use DNAweaver to build oligos """
    # Here we use a comercial supplier but don't actually care. 
    cheap_dna_offer = dw.CommercialDnaOffer(
        name="CheapDNA.",
        sequence_constraints=[
            dw.NoPatternConstraint(enzyme="BsaI"),
            dw.SequenceLengthConstraint(max_length=4000),
            dw.GcContentConstraint(min_gc=min_gc, max_gc=max_gc)
        ],
        pricing=dw.PerBasepairPricing(0.10),
    )

    oligo_dna_offer = dw.CommercialDnaOffer(
        name="OliGoo",
        sequence_constraints=[
            dw.GcContentConstraint(min_gc=min_gc, max_gc=max_gc),
            dw.SequenceLengthConstraint(max_length=4000),
        ],
        pricing=dw.PerBasepairPricing(0.07),
        memoize=True
    )

    oligo_assembly_station = dw.DnaAssemblyStation(
        name="Oligo Assembly Station",
        assembly_method=dw.OligoAssemblyMethod(
            overhang_selector=dw.TmSegmentSelector(
                min_size=15, max_size=25, min_tm=min_tm, max_tm=max_tm
            ),
            min_segment_length=min_segment_length,
            max_segment_length=max_segment_length,
            sequence_constraints=[dw.SequenceLengthConstraint(max_length=4000)],
            duration=8,
            cost=30,
        ),
        supplier=oligo_dna_offer,
        coarse_grain=20,
        a_star_factor="auto",
        memoize=True,
    )

    assembly_station = dw.DnaAssemblyStation(
        name="Gibson Assembly Station",
        assembly_method=dw.GibsonAssemblyMethod(
            overhang_selector=dw.TmSegmentSelector(min_tm=min_tm, max_tm=max_tm),
            min_segment_length=min_segment_length,
            max_segment_length=max_segment_length + 20, # add a bit of a buffer
        ),
        supplier=[cheap_dna_offer, oligo_assembly_station],
        logger="bar",
        coarse_grain=100,
        fine_grain=10,
        a_star_factor="auto",
    )
    
    print("Looking for the best assembly plan...")
    t0 = time.time()
    quote = assembly_station.get_quote(sequence, with_assembly_plan=True)
    assembly_plan_report = quote.to_assembly_plan_report()
    assembly_plan_report.write_full_report(f"{output_directory}/oligo_assembly_plan_{seq_id}.zip")
    original_sequence = assembly_plan_report.plan.sequence
    # Then get the sequence 
    rows = []
    for oligo in assembly_plan_report.plan.assembly_plan:
        # If this was chosen then choose it
        if oligo.accepted:
            rows.append([oligo.id, oligo.sequence, original_sequence])
    return rows

def get_oligos(df, protein_column, id_column, output_directory, forward_primer: str, reverse_primer: str, sequence_end: str, min_overlap=10, min_gc=0.3, 
               max_gc=0.7, min_tm=55, max_tm=70, min_segment_length=90, max_segment_length=130, max_length=1500, genbank_file=None,
               insert_position=0, simple=False, optimize=True, bsa=False):
    """ Get the oligos for a dataframe:
    sequence_end is the end of the sequence i.e. TAA, TGA, etc or a histag 
    """
    rows = []   
    record = None
    for seq_id, protein_sequence in df[[id_column, protein_column]].values:
        # Add on the primers that the user has provided
        if optimize:
            optimzed_sequence = codon_optimize(protein_sequence, min_gc, max_gc) + sequence_end
        else:
            optimzed_sequence = protein_sequence + sequence_end
            u.dp([f"Added sequence end with HIS and stop to: {seq_id}, {sequence_end}"])

        if genbank_file:
            # Add in the optimzed sequence
            translation_label = f"Insert_{seq_id}"
            reverse = False  # Set to True for reverse feature, False for forward
            record = insert_sequence_with_translation(genbank_file, None, insert_position, optimzed_sequence, translation_label, reverse)
            
        if optimzed_sequence[:3] != "ATG":
            u.dp([f"Warning: {seq_id} does not start with a methionine. ", optimzed_sequence[:3]])
            if 'ATG' not in forward_primer:
                u.warn_p([f"Warning: {seq_id} does not start with a methionine. AND you don't have a methonine in your primer!!", forward_primer])
                print("We expect the primer to be in 5 to 3 prime direction.")
        # ALso check the end and or the reverse primer check for the three ones
        if optimzed_sequence[-3:] != "TAA" and optimzed_sequence[-3:] != "TGA" and optimzed_sequence[-3:] != "TAG":
            u.dp([f"Warning: {seq_id} does not end with a stop codon. ", optimzed_sequence[-3:]])
            if 'TAA' not in reverse_primer and "TGA" not in reverse_primer and "TAG" not in reverse_primer:
                u.warn_p([f"Warning: {seq_id} does not end with a stop codon. AND you don't have a stop codon in your primer!!", reverse_primer])
                print("We expect the primer to be in 5 to 3 prime direction.")
        codon_optimized_sequence = forward_primer + optimzed_sequence + reverse_primer
        #try:
        # Check now some simple things like that there is 
        if simple:
            oligos = build_simple_oligos(seq_id, codon_optimized_sequence, min_segment_length, max_segment_length)
        else:
            oligos = build_oligos(seq_id, codon_optimized_sequence, output_directory, min_gc, max_gc, min_tm, max_tm, min_segment_length, max_segment_length, max_length)
        # except Exception as e:
        #     u.warn_p([f"Warning: {seq_id} did not have any oligos built. ", e])
        #     oligos = []
        prev_oligo = None
        # If a genbank file was provided also just add in the new sequnece
        if len(oligos) > 0:
            for i, oligo in enumerate(oligos):
                seq = oligo[1]
                # CHeck that there is an overlap with the previous sequence and that it is not too short
                # Also make sure we swap the directions of the oligos so they automatically anneal
                # Also assert that the start is a methionine (and if not warn it... )
                primer_overlap = None
                primer_tm = None
                primer_len = None
                homodimer_tm = None
                hairpin_tm = None
                if prev_oligo:  
                    # Get the overlap with the previous sequence
                    match = SequenceMatcher(None, prev_oligo, seq).find_longest_match()
                    primer_overlap = prev_oligo[match.a:match.a + match.size]
                    # Analyze the primer sequence
                    results = check_secondary_structure(primer_overlap)
                    homodimer_tm = results['homodimer']['homodimer_dg']
                    hairpin_tm = results['hairpin']['hairpin_dg']
                    primer_tm = primer3.bindings.calcTm(primer_overlap)
                    primer_len = len(primer_overlap)

                prev_oligo = seq
                orig_seq = seq
                strand = 1
                # Changed this to be the opposite To
                if bsa == True:
                    if i % 2 == 1 and i != 0:
                        seq = str(Seq(seq).reverse_complement())
                        strand = -1
                else:
                    if i % 2 == 0:
                        seq = str(Seq(seq).reverse_complement())
                        strand = -1
                oligo_tm = primer3.bindings.calcTm(seq)
                if genbank_file:
                    insert_features_from_oligos(record, f"{seq_id}_oligo_{i}", orig_seq, strand, oligo_tm, None)
                rows.append([seq_id, oligo[0], seq, len(seq), oligo_tm, strand, primer_overlap, primer_tm, primer_len, homodimer_tm, hairpin_tm, oligo[2]])
            if genbank_file:
                output_file = genbank_file.replace('.', f'_{seq_id}.')
                if genbank_file:
                    record.name = seq_id  # type: ignore 
                    SeqIO.write(record, f'{output_directory}/{output_file}', "genbank")
        else:
            u.warn_p([f"Warning: {seq_id} did not have any oligos built. ", optimzed_sequence])
            rows.append([seq_id, None, optimzed_sequence, len(optimzed_sequence), None, None, None, None, None, None, None, None])
        
    oligo_df = pd.DataFrame(rows, columns=["id", "oligo_id", "oligo_sequence", "oligo_length",  "oligo_tm", "strand","primer_overlap_with_previous", "overlap_tm_5prime", "overlap_length", 
                                            "overlap_homodimer_dg", "overlap_hairpin_dg", "original_sequence"])
    
        
    return oligo_df


def build_simple_oligos(seq_id: str, sequence: str, min_segment_length=90, max_segment_length=130, overlap_len=18, optimal_temp=62, overlap_buffer=10):
    # Basically get the splits size
    seq_len = len(sequence)
    num_splits = int(math.ceil(seq_len / min_segment_length))
    # Make sure it is even
    if num_splits % 2 == 0:
        num_splits += 1
    remainder = seq_len % num_splits
    print(num_splits, remainder)        
    # Now we want to go through the new part length and while remainder is greater then 0 we distribute this across the splits
    max_part_len = math.floor(seq_len/num_splits)
    split_counts = {}
    for i in range(0, num_splits + 1):
        split_counts[i] = max_part_len
    # Now distribute the remainder
    split_count = 0
    for i in range(0, remainder + 1):
        split_counts[split_count] += 1
        split_count += 1
        # Iterate through this again
        if split_count == num_splits + 1:
            split_count = 0
    prev_cut = 0
    rows = []
    finished = False
    for i in split_counts:
        part_len = split_counts[i]
        cut = prev_cut + part_len
        oligo = sequence[prev_cut:cut]
        # Calculate the tm and we'll check that we get the "best" one i.e. closest to 62 deg
        # Get the overlap with the previous sequence
        best_oligo = oligo
        best_tm_diff = 10000
        best_cut = cut
        part_len_diff = 0
        best_pl = 0 # The buffer of the overlap - was 10 by default
        for pl in range(overlap_buffer, 0, -1):
            for j in range(0, max_segment_length - min_segment_length):
                cut = prev_cut + part_len + pl + j
                oligo = sequence[prev_cut:cut]
                primer_overlap = oligo[-1 * (overlap_len + pl):]
                # Analyze the primer sequence
                primer_tm = (primer3.bindings.calcTm(primer_overlap) + primer3.bindings.calcTm(str(Seq(primer_overlap).reverse_complement())))/2
                results = check_secondary_structure(primer_overlap)
                # Want to have the opp a high homodimer TM
                homodimer_tm = -1 * results['homodimer']['homodimer_dg']
                # Since the homodimer is negative and we're trying to minimize invert this.
                if homodimer_tm < 1:
                    # Ignore this unless it's actually a problem (otherwise we want to just care about temperature)
                    homodimer_tm = 0
                if (abs(primer_tm - optimal_temp) + homodimer_tm) < best_tm_diff:
                    best_tm_diff = abs(primer_tm - optimal_temp) + homodimer_tm
                    best_oligo = oligo
                    best_cut = cut
                    part_len_diff = j + pl  # Add the new offset and the overlap buffer (here we allow 18 to 29)
                    best_pl = pl
        # check the left over size
        if len(sequence[best_cut:]) < 30:
            # Add on the last bit and just have a longer final oligo
            rows.append([f'{seq_id}_{i}', best_oligo + sequence[best_cut:], sequence, prev_cut, best_cut + len(sequence[best_cut:]), part_len + part_len_diff, 'last'])
            finished = True
            break
        rows.append([f'{seq_id}_{i}', best_oligo, sequence, prev_cut, best_cut, part_len + part_len_diff, 'normal'])
        prev_cut = best_cut - overlap_len - best_pl
    # Add in the last one
    oligo = sequence[prev_cut:]
    if not finished:
        part_len = len(oligo)
        cut = len(sequence)
        if len(oligo) < 18:
            u.warn_p(["Last oligo very short,.... check this!", f'{seq_id}_{split_counts + 1}', oligo, sequence, prev_cut, cut, part_len, 'last'])
        print(prev_cut, part_len, len(sequence), oligo)
        rows.append([f'{seq_id}_{split_counts}', oligo, sequence, prev_cut, cut, part_len, 'last'])
    return rows


def codon_optimize(protein_sequence: str, min_gc=0.3, max_gc=0.7):
    """ Codon optimize the protein sequence using DNA chisel: https://github.com/Edinburgh-Genome-Foundry/DnaChisel"""
    seq = dnachisel.reverse_translate(protein_sequence)
    problem = dnachisel.DnaOptimizationProblem(
        sequence=seq,
        constraints=[
            AvoidPattern("BsaI_site"), # type: ignore
            EnforceGCContent(mini=min_gc, maxi=max_gc, window=50), # type: ignore
            EnforceTranslation(location=(0, len(seq))), # type: ignore
            AvoidStopCodons(location=(0, len(seq)-3)) # type: ignore Let there be stop codons in the last bit
        ],
        objectives=[CodonOptimize(species='e_coli', location=(0, len(seq)))] # type: ignore
    )
    # SOLVE THE CONSTRAINTS, OPTIMIZE WITH RESPECT TO THE OBJECTIVE
    problem.resolve_constraints()
    problem.optimize()

    # PRINT SUMMARIES TO CHECK THAT CONSTRAINTS PASS
    print(problem.constraints_text_summary())
    print(problem.objectives_text_summary())

    # GET THE FINAL SEQUENCE (AS STRING OR ANNOTATED BIOPYTHON RECORDS)
    final_sequence = problem.sequence  # string
    final_record = problem.to_record(with_sequence_edits=True)
    print(protein_sequence)
    print(final_sequence)
    return final_sequence

In [11]:
import numpy as np
import pyswarms as ps
from primer3 import bindings
from functools import partial
import math

# --- Parameters ---
min_len, max_len = 80, 130
min_olap, max_olap = 18, 27
tm_target = 62

def get_tm(seq):
    try:
        return bindings.calcTm(seq)
    except:
        return 100  # Penalize invalid sequences

def compute_fragments(cut_sites, seq):
    cut_sites = np.sort(np.round(cut_sites).astype(int))
    cut_sites = [0] + list(cut_sites) + [len(seq)]

    fragments = []
    overlaps = []

    for i in range(len(cut_sites)-1):
        frag = seq[cut_sites[i]:cut_sites[i+1]]
        fragments.append(frag)
        if i < len(cut_sites) - 2:
            # Define overlap with next fragment
            overlap = seq[cut_sites[i+1]-max_olap : cut_sites[i+1]]
            overlaps.append(overlap)

    return fragments, overlaps

def objective_function(x, seq):
    penalties = []
    for particle in x:
        particle = np.clip(particle, 0, len(seq))  # Ensure inside sequence
        fragments, overlaps = compute_fragments(particle, seq)

        total_penalty = 0
        for frag in fragments:
            if not (min_len <= len(frag) <= max_len):
                total_penalty += 100 * abs(len(frag) - (min_len + max_len)/2)

        for olap in overlaps:
            if not (min_olap <= len(olap) <= max_olap):
                total_penalty += 100
            tm = get_tm(olap)
            total_penalty += abs(tm - tm_target) * 10

        penalties.append(total_penalty)
    return np.array(penalties)

# Example: Optimize cut sites for a single sequence
def optimize_sequence(seq, num_fragments):
    num_cuts = num_fragments - 1
    seq_len = len(seq)

    # Approx evenly spaced initial bounds (naive)
    margin = max_len + max_olap
    lower_bound = np.array([i * min_len for i in range(1, num_fragments)])
    upper_bound = np.array([seq_len - (num_fragments - i) * min_len for i in range(1, num_fragments)])

    bounds = (lower_bound, upper_bound)
    print(bounds)
    
    # optimizer = ps.single.GlobalBestPSO(
    #     n_particles=50,
    #     dimensions=num_cuts,
    #     options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
    #     bounds=bounds
    # )

    # best_cost, best_pos = optimizer.optimize(partial(objective_function, seq=seq), iters=200)
    # return best_cost, best_pos



In [12]:
cost, positions = optimize_sequence('gaaataattttgtttaactttaagaaggagatatacatATGCGCAAGCTTTCTCGTTGTGAGGACGGCTACGTCAAGATTCAGGGGATCTACATTTACTATAAAGTGTGCAAAGCCGAGAATGAGAAAGCCAAGTTAATGACACTGCACGGTGGACCCGGCATGAGCCACGACTACTTGCTTTCCCTGACCGATTTAGCTGAGAAGGGGATTACGGTTTTATTCTATGACCAATTCGGCTGTGGACGTTCAGAAGAACCTGAGAAGGAGAAATTTACGATCGACTATGGAGTGGAAGAAGCAGAAGCGGTGAAAAAAAACATCTTTGGTGATGATAAGGTATTCCTGATGGGATCGTCCTATGGTGGGGCTTTAGCTCTGGCCTACGCTGTTAAGTATCAAGCACATTTGAAGGGCCTGATCATCTCGGGCGGATTATCATCTGTGCCTCTGACAGTAAAAGAAATGCAGCGCTTGATTGACGAGCTTCCAGAAAAGTACCGCAACGCGATCCGCAAGTATGGCGAAGTGGGGGATTATCAGAACCCTGAATACCAGGAGGCTGTGAACTATTTCTATCACCAGCATTTGTTACGCTCTGAAGACTGGCCCCCCGAGGTTTTGAAAAGCCTGGAGTACGCAGAAGAGCGCAATGTGTACCGCACAATGAACGGCCCCAATGAATTTACAATCACTGGAACTATCCGCGACTGGGATATCACCGATAAGATCGGGATTATCAGTGTTCCTACTCTTATTACGGTCGGTGAGTTTGACGAGGTAACGCAAAATGTCGCAGAAGTTATCCACTCCAAAATCGATAATTCCCAGTTAATTGTATTTAAAGCATGTAGCCACCTGACCATGTGGGAGGATCGTGATGAGTACAACCGCATTTTATTACAGTTCATCGAAAAGAACATTCTCGAGCACCACCACCACCACCACTGAgatccggc', 17)

(array([  80,  160,  240,  320,  400,  480,  560,  640,  720,  800,  880,
        960, 1040, 1120, 1200, 1280]), array([-322, -242, -162,  -82,   -2,   78,  158,  238,  318,  398,  478,
        558,  638,  718,  798,  878]))


TypeError: cannot unpack non-iterable NoneType object

In [9]:
positions.sort()
positions = [int(p) for p in positions]
oligo

[102,
 214,
 293,
 319,
 399,
 403,
 430,
 519,
 535,
 615,
 639,
 668,
 754,
 772,
 783,
 862]